In [1]:
!pip install mediapipe opencv-python pandas 

In [2]:
!pip install scikit-learn==1.5.1

In [3]:
import sklearn
print(sklearn.__version__)

1.5.1


In [4]:
import mediapipe as mp
import cv2
import numpy as np

In [5]:
VIDEO_PATH='hips.avi'
EXPORT_PATH='hips.csv'
MODEL_PATH='hips.pkl'

In [6]:
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [12]:
cap=cv2.VideoCapture(0)

height= cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps=cap.get(cv2.CAP_PROP_FPS)
videoWriter=cv2.VideoWriter(VIDEO_PATH,cv2.VideoWriter_fourcc('P','I','M','1'),fps,(int(width),int(height)))

while cap.isOpened():
    ret,frame=cap.read()

    try:
        cv2.imshow('videoHips',frame)
        videoWriter.write(frame)
    except Exception as e:
        break

    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

In [7]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [25]:
landmarks=['class']
for val in range(1,33+1):
    landmarks +=['x{}'.format(val),'y{}'.format(val),'z{}'.format(val),'v{}'.format(val)]

In [10]:
with open(EXPORT_PATH,mode='w',newline='') as f:
    csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [11]:
def export_landmark(results,action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0,action)
        with open(EXPORT_PATH,mode='a',newline='') as f:
            csv_writer=csv.writer(f,delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass

In [13]:
cap=cv2.VideoCapture(VIDEO_PATH)

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret,image=cap.read()

        #Recolor Feed 
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        #Make desicions
        results=pose.process(image)
        #Recolor image vack to bgr for rendering
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_pose.POSE_CONNECTIONS
                              , mp_drawing.DrawingSpec(color=(2.45,117,66), thickness=2, circle_radius=4),
                               mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                        )
        k=cv2.waitKey(1)
        if k==ord('a'): 
            export_landmark(results,'wide')
        if k==ord('s'): 
            export_landmark(results,'neutral')
        if k==ord('d'): 
            export_landmark(results,'narrow')

        cv2.imshow('Raw Webcam Feed',image)

        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
cap.release()
cv2.destroyAllWindows()

# Train

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [9]:
df=pd.read_csv(EXPORT_PATH)

In [10]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,wide,0.472998,0.354487,-0.327363,0.999913,0.478231,0.341720,-0.313608,0.999751,0.482576,...,0.174157,0.910317,0.635675,0.988030,-0.015988,0.980426,0.376225,0.993535,0.039443,0.982520
1,wide,0.494815,0.356109,-0.381467,0.999943,0.499105,0.343205,-0.361872,0.999837,0.502743,...,0.256465,0.905037,0.629246,0.993659,-0.082558,0.982512,0.379748,0.989783,0.129210,0.983554
2,wide,0.497151,0.356078,-0.371119,0.999947,0.501747,0.343200,-0.353578,0.999850,0.505397,...,0.208290,0.896944,0.630956,0.994192,-0.075676,0.982099,0.389123,0.989401,0.071118,0.982753
3,wide,0.497433,0.356305,-0.398738,0.999951,0.503592,0.343440,-0.383563,0.999870,0.507901,...,0.415745,0.834012,0.622942,0.993654,-0.218309,0.976719,0.401621,0.924198,0.314368,0.964572
4,wide,0.496348,0.358616,-0.263019,0.999726,0.501768,0.345610,-0.245537,0.999372,0.506272,...,0.010525,0.479552,0.640685,1.020189,-0.127493,0.826451,0.365216,0.934905,-0.111190,0.633458


In [11]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
157,narrow,0.498397,0.339488,-0.353637,0.999866,0.504233,0.326848,-0.340208,0.999535,0.508471,...,0.237902,0.725516,0.510151,0.977424,0.028844,0.888612,0.477342,0.970934,0.119213,0.888941
158,narrow,0.496314,0.339986,-0.344266,0.999884,0.502487,0.327255,-0.330382,0.999597,0.506687,...,0.251239,0.764158,0.508561,0.973953,0.039694,0.891114,0.476047,0.966487,0.135660,0.898936
159,narrow,0.495051,0.341595,-0.347796,0.999885,0.501209,0.328573,-0.333475,0.999598,0.505482,...,0.193860,0.759699,0.508655,0.978762,-0.009734,0.887980,0.476358,0.967652,0.073664,0.896564
160,narrow,0.490597,0.341434,-0.355969,0.999881,0.496984,0.328546,-0.342164,0.999581,0.501536,...,0.202272,0.744213,0.512099,0.987469,0.010470,0.885892,0.477212,0.975118,0.075575,0.893980
161,narrow,0.484512,0.341224,-0.369713,0.999870,0.490787,0.328389,-0.355269,0.999575,0.495522,...,0.229063,0.713351,0.518258,0.987414,0.012324,0.863908,0.473799,0.979939,0.101938,0.855277


In [12]:
X=df.drop('class',axis=1)
y=df['class']

In [13]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1234)

In [14]:
y_test

149     narrow
145    neutral
40      narrow
63        wide
122    neutral
140       wide
39      narrow
88     neutral
97      narrow
6       narrow
148     narrow
109       wide
157     narrow
27      narrow
25      narrow
137       wide
42      narrow
9         wide
150     narrow
65        wide
58        wide
51        wide
61        wide
29      narrow
35      narrow
133    neutral
11        wide
126    neutral
48        wide
99      narrow
17     neutral
156     narrow
93      narrow
66        wide
119    neutral
57        wide
100     narrow
54        wide
77      narrow
73      narrow
130    neutral
117    neutral
74      narrow
159     narrow
91      narrow
43      narrow
114     narrow
135    neutral
79        wide
Name: class, dtype: object

# train classify model 

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [16]:
pipelines={
    'rf': make_pipeline(StandardScaler(),RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(),GradientBoostingClassifier()),
}

In [17]:
fit_models={}
for algo, pipeline in pipelines.items():
    model=pipeline.fit(X_train,y_train)
    fit_models[algo]=model

In [18]:
fit_models

{'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [19]:
fit_models['rf'].predict(X_test)

array(['narrow', 'neutral', 'narrow', 'wide', 'neutral', 'wide', 'narrow',
       'neutral', 'narrow', 'narrow', 'narrow', 'wide', 'narrow',
       'narrow', 'narrow', 'wide', 'narrow', 'wide', 'narrow', 'wide',
       'wide', 'wide', 'wide', 'narrow', 'narrow', 'neutral', 'wide',
       'neutral', 'wide', 'narrow', 'neutral', 'narrow', 'narrow', 'wide',
       'neutral', 'wide', 'narrow', 'wide', 'narrow', 'narrow', 'neutral',
       'neutral', 'narrow', 'narrow', 'narrow', 'narrow', 'narrow',
       'neutral', 'wide'], dtype=object)

# Evaluate 

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score #Accuracy metrics
import pickle

In [21]:
for algo,model in fit_models.items():
    yhat=model.predict(X_test)
    print(algo, accuracy_score(y_test.values,yhat),
         precision_score(y_test.values,yhat,average="weighted"),
         recall_score(y_test.values,yhat,average="weighted",))

rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


In [22]:
 with open(MODEL_PATH,'wb')as f:
     pickle.dump(fit_models['rf'],f)

# Make detections


In [23]:
with open(MODEL_PATH,'rb') as f:
    model=pickle.load(f)

In [26]:
# Inicializar mediapipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Inicializar el modelo de VideoCapture
cap = cv2.VideoCapture(0)

counter = 0
current_stage = ''

# Inicializar el modelo de pose
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolorear el feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Hacer decisiones
        results = pose.process(image)

        # Recolorear la imagen de nuevo a BGR para renderizar
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Dibujar las landmarks de la pose
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            try:
                # Obtener las landmarks
                row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                
                # Asegúrate de definir 'landmarks' y 'model' en tu contexto
                X = pd.DataFrame([row], columns=landmarks[1:])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
               
                print(body_language_class, body_language_prob)

                if body_language_class == 'down' and body_language_prob[np.argmax(body_language_prob)] >= .7:
                    current_stage = 'down'
                elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[np.argmax(body_language_prob)] <= .7:
                    current_stage = "up"
                    counter += 1
                    print(current_stage)

                # Dibujar los cuadros de las etiquetas
                cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

                cv2.putText(image, 'CLASS', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0], (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Mostrar la probabilidad
                cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)), (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                
            except Exception as e:
                print(f"Error: {e}")
                pass

        cv2.imshow('Raw Webcam Feed', image)
        # Captura la tecla 'r' para resetear el contador y el estado
        if cv2.waitKey(10) & 0xFF == ord('r'):
            counter = 0
            current_stage = ''
            print("Reset done!")

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

narrow [0.38 0.25 0.37]
narrow [0.38 0.25 0.37]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


narrow [0.38 0.25 0.37]
narrow [0.38 0.25 0.37]
narrow [0.38 0.24 0.38]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\pro

narrow [0.39 0.24 0.37]
narrow [0.38 0.25 0.37]
wide [0.37 0.25 0.38]


...

wide [0.19 0.31 0.5 ]
wide [0.3  0.27 0.43]
wide [0.3  0.27 0.43]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


wide [0.3  0.27 0.43]
wide [0.27 0.28 0.45]
wide [0.21 0.31 0.48]


C:\Users\Alex\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


wide [0.33 0.28 0.39]
